In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [2]:
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from utilities import aggregate_feature_calculators
from utilities import aggregate_feature_calculators_setting as aggcal
from utilities.parallel import Parallel
from utilities.dfdb import DFDB

from utilities.process.pqueue import *
from utilities.process.pnode import *
from utilities.process.putilities import *

Using TensorFlow backend.


In [3]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import lightgbm as lgb
import catboost as cb
from keras.wrappers.scikit_learn import KerasRegressor
from keras import Sequential
from keras.layers import Dense

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
import optuna

# from common import EP
# from models import *
# from dfdb import DFDB

import types
import os
import copy

import seaborn as sns
import matplotlib.pyplot as plt

# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# os.environ["CUDA_VISIBLE_DEVICES"]="3";  

In [4]:
class KerasMLPRegressor(object):
    
    def __init__(self, batch, input_dim, hidden_layer_sizes, activation, dropout, l1l2regularizer, solver, metric, lr, sgd_momentum, sgd_decay, base_save_dir, alias):
        
        self.batch = batch
        self.input_dim = input_dim
        self.hidden_layer_sizes = hidden_layer_sizes
        self.activation = activation
        self.solver = solver
        self.metric = metric
        self.dropout = dropout
        self.l1l2regularizer = l1l2regularizer
        self.lr = lr
        self.sgd_momentum = sgd_momentum
        self.sgd_decay = sgd_decay
        
        self.regressor = self.build_graph(input_dim, hidden_layer_sizes, activation, dropout, l1l2regularizer)
        self.compile_graph(self.regressor, solver, metric, lr, sgd_momentum, sgd_decay)
        
        self.alias = alias
        self.base_save_dir = base_save_dir
        if (self.alias==None) & (self.base_save_dir==None):
            self.chkpt = None
        else:
            self.chkpt = os.path.join(base_save_dir,'{}.hdf5'.format(alias))

        return
    
    def build_graph(self, input_dim, hidden_layer_sizes, activation, dropout, l1l2regularizer):
        
        if type(l1l2regularizer) == type(None):
            regularizer=None
        else:
            regularizer = regularizers.l1_l2(l1l2regularizer)
    
        i = Input(shape = (input_dim,))
        x = Dense(hidden_layer_sizes[0], activation=activation, kernel_regularizer=regularizer, activity_regularizer=regularizer)(i)
        x = BatchNormalization()(x)
        x = Dropout(dropout)(x)
        for units in hidden_layer_sizes[1:-1]:
            x = Dense(units, activation=activation, kernel_regularizer=regularizer, activity_regularizer=regularizer)(x)
            x = BatchNormalization()(x)
            x = Dropout(dropout)(x)
        x = Dense(hidden_layer_sizes[-1], activation=activation, kernel_regularizer=regularizer, activity_regularizer=regularizer)(x)
        x = BatchNormalization()(x)
        y = Dense(1)(x)
        regressor = Model(inputs = [i], outputs = [y])
        return regressor
    
    def compile_graph(self, model, solver, metric, lr, momentum, decay):
        if solver=='adam':
            optimizer = optimizers.adam(lr=lr)
        elif solver=='sgd':
            optimizer = optimizers.SGD(lr=lr, decay=decay, momentum=momentum, nesterov=True)
        model.compile(optimizer=optimizer, loss=metric)
        return
    
    def fit(self, X_train, y_train, eval_set, versbose=1, epochs=200, early_stopping_rounds=20):
        
#         reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=early_stopping_rounds//4, min_lr=self.lr*1e-2)
        es_cb = EarlyStopping(monitor='val_loss', patience=early_stopping_rounds, verbose=1, mode='auto')
        cp_cb = ModelCheckpoint(filepath = self.chkpt, monitor='val_loss', verbose=versbose, save_best_only=True, mode='auto')

#         his_train = self.regressor.fit_generator( generator =  train_gen, epochs = epochs,  verbose = 1,  validation_data = validation, callbacks = [cp_cb])
        his_train = self.regressor.fit( X_train, y_train, epochs = epochs,  verbose = versbose,  validation_data = eval_set[0], callbacks = [])
        df_train_his = pd.DataFrame(his_train.history)
        
#         df_train_his = pd.DataFrame()
#         prev_val_loss = 999999
#         for i in np.arange(epochs):
#             his_train = self.regressor.fit( X_train, y_train, epochs = 1,  verbose = versbose,  batch_size = self.batch,  validation_data = validation,  callbacks = [])
#             df_train_his_i = pd.DataFrame(his_train.history)
#             df_train_his_i['epochs'] = i+1
#             df_train_his = pd.concat([df_train_his, df_train_his_i], axis=0)
#             if (df_train_his_i.val_loss.values[0] < prev_val_loss) & (self.chkpt!=None):
#                 prev_val_loss = df_train_his_i.val_loss.values[0]
#                 self.regressor.save_weights(self.chkpt)
                
        df_train_his.to_csv(self.base_save_dir + '/{}_train_his.csv'.format(self.alias), index=True)
            
        return df_train_his
    
    def predict(self, X, use_best_epoch=False):
        if use_best_epoch:
            self.regressor.load_weights(self.chkpt)
        return self.regressor.predict(X)[:,0]

In [5]:
file_folder =  '../../data/feature'
[f for f in os.listdir(file_folder) if not f.startswith('.')]

['angles-and-distances_test.pkl',
 'angles-and-distances_train.pkl',
 'brute-force-feature-engineering_test.pkl',
 'eem_test.pkl',
 'giba-r-data-table-simple-features-0-991-lb_test.pkl',
 'giba-r-data-table-simple-features-0-991-lb_train.pkl',
 'brute-force-feature-engineering_train.pkl',
 'keras-neural-net-for-champs_test.pkl',
 'keras-neural-net-for-champs_train.pkl',
 'eem_train.pkl',
 'molecular-properties-eda-and-models_test.pkl',
 'molecular-properties-eda-and-models_train.pkl',
 'molecule-with-openbabel_test.pkl',
 'molecule-with-openbabel_train.pkl',
 'dataset-with-number-of-bonds-between-atoms_test.pkl',
 'brute-force-feature-engineering-mini_test.pkl',
 'df_train.gzde',
 'df_test.gzde',
 'eachtype_train.pkl',
 'eachtype_test.pkl',
 'giba-r-data-table-simple-features-1-17-lb_test.pkl',
 'dataset-with-number-of-bonds-between-atoms_train.pkl',
 'brute-force-feature-engineering-mini_train.pkl',
 'pytorch_geometric_test.pkl',
 'pytorch_geometric_train.pkl',
 'giba-r-data-table-sim

In [6]:
df_train=pd.read_pickle(f'{file_folder}/tmp_df_train', compression='gzip')
df_test=pd.read_pickle(f'{file_folder}/tmp_df_test', compression='gzip')
df_train['y'] = df_train['scalar_coupling_constant']

In [7]:
def create_path(base_dir, param):
    if base_dir == None:
        return None
    fold_path = base_dir + '/' + ','.join("{!s}={!r}".format(key,val) for (key,val) in param.items())
    if not os.path.exists(fold_path):
        os.makedirs(fold_path)
    return fold_path

path_param={
    'hidden_layer_sizes':[64, 16],
    'activation':'relu',
    'l1l2regularizer':None,
    'dropout':.3,
}
base_save_dir = create_path('KerasMLPRegressor', path_param)
param={
    'columns': [],
 'cv': {'cls': 'KFold',
  'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}},
 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}},
 'model': {'cls': 'KerasMLPRegressor',
  'init': {'input_dim':0,
            'batch':128, 
            'solver':'adam', 
            'metric':'mean_absolute_error', 
            'lr':.0001, 
            'sgd_momentum':.9, 
            'sgd_decay':0.0001,
            'base_save_dir':base_save_dir, 
            'alias':'kerasmlp',
            **path_param},
  'fit':  {'versbose':0, 
            'epochs':50, 
            'eval_set':{}},},
 'metric': 'mean_absolute_error'
}
    

In [8]:
columns_list = [
{'columns': ['tertiary_distance_2', 'dist_H_2_y', 'dist_H_0_y', 'distC0', 'inv_dist1R', 'dist_C_2_x', 'dist_to_type_std', 'dist_C_2_y', 'dist_C_0_x', 'atom_1_n_bonds', 'd_5_1', 'yukawa_H.y', 'd_4_1', 'd_4_2', 'd_4_3', 'd_3_0', 'dist_H_2_x', 'd_4_0', 'tertiary_distance_4', 'tertiary_angle_1', 'd_3_1', 'dist_H_1_y', 'coulomb_H.y', 'dist_C_4_x', 'atom_index_farthest_0', 'distance_farthest_0', 'tertiary_angle_2', 'distance_c1', 'atom_1_bond_lengths_std', 'inv_dist0R', 'eem_0', 'inv_distPR', 'dist_N_0_y', 'dist_H_3_x', 'distC1', 'tertiary_atom_0', 'dist_H_1_x', 'dist_C_0_y', 'adC1', 'max_distance_y', 'tertiary_distance_5', 'dist_to_type_0_mean', 'atom_1_bond_lengths_mean', 'd_1_0', 'dist_O_1_y', 'tertiary_angle_0', 'yukawa_N.x', 'dist_C_4_y', 'atom_index_1_cycle_size_mean', 'tertiary_atom_1', 'inv_dist1', 'd_2_1', 'dist_C_1_y', 'adC2', 'inv_dist1E', 'molecule_atom_index_1_dist_min_diff', 'dist_to_type_1_mean', 'dist_to_type_mean', 'adC3', 'dist_O_0_y', 'eem_1', 'adN1', 'tertiary_distance_3', 'dist_N_0_x', 'molecule_atom_index_0_dist_max_div', 'dist_O_1_x', 'dist_C_3_y', 'tertiary_angle_3', 'cos_f0', 'd_3_2', 'dist_O_0_x', 'dist_H_3_y', 'dist_C_3_x', 'tertiary_angle_4'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_estimators':3000, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['yukawa_N.x', 'inv_distPR', 'molecule_atom_index_0_dist_mean_div', 'dist_C_0_y', 'dist_N_1_x', 'dist_C_1_x', 'd_4_1', 'molecule_atom_1_dist_min_diff', 'dist_N_0_y', 'cos_f1', 'dist_O_0_x', 'tertiary_distance_5', 'vander_N.x', 'dist_C_3_x', 'coulomb_H.x', 'dist_C_3_y', 'd_9_2', 'distance_center0', 'd_3_2', 'd_5_2', 'dist_C_2_x', 'dist_to_type_0_mean', 'dist_N_0_x', 'dist_O_0_y', 'd_4_3', 'dist_H_1_x', 'atom_1_bond_lengths_max', 'atom_1_bond_lengths_std', 'd_3_1', 'dist_H_3_y', 'tertiary_distance_1', 'molecule_atom_index_0_dist_max_div', 'inv_distP', 'tertiary_angle_3', 'tertiary_distance_0', 'linkN', 'd_2_0', 'd_4_2', 'd_5_3', 'dist_to_type_std', 'd_4_0', 'd_3_0', 'dist_H_0_y', 'dist_C_0_x', 'tertiary_angle_0', 'dist_N_1_y', 'adC3', 'inv_dist1R', 'dist_H_1_y', 'eem_1', 'dist_C_1_y', 'tertiary_distance_3', 'd_2_1', 'dist_H_2_y', 'tertiary_distance_4', 'link0', 'tertiary_angle_2', 'd_1_0', 'dist_C_2_y', 'tertiary_atom_0', 'yukawa_H.y', 'molecule_name.1', 'distance_c1', 'inv_dist0R', 'max_molecule_atom_0_dist_xyz', 'atom_1_bond_lengths_mean', 'cos_center0', 'cos_center0_center1', 'inv_dist1', 'eem_0', 'vander_H.x', 'tertiary_distance_2', 'dist_C_4_x', 'distN0', 'dist_H_0_x', 'max_distance_y', 'cos_f0'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_estimators':3000, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['molecule_atom_index_1_dist_min_diff', 'tertiary_angle_1', 'dist_C_0_y', 'tertiary_atom_2', 'd_6_2', 'dist_C_1_x', 'atom_8', 'd_4_1', 'd_9_1', 'dist_N_0_y', 'cos_c1', 'd_7_0', 'dist_O_0_x', 'tertiary_distance_5', 'dist_C_3_x', 'dist_O_1_x', 'dist_C_3_y', 'd_7_2', 'd_3_2', 'd_5_2', 'd_6_0', 'dist_C_2_x', 'dist_N_0_x', 'dist_to_type_0_mean', 'molecule_atom_index_0_dist_max_diff', 'dist_O_0_y', 'd_4_3', 'dist_H_1_x', 'atom_1_bond_lengths_std', 'd_3_1', 'd_5_0', 'tertiary_distance_1', 'molecule_atom_index_0_dist_max_div', 'adC1', 'tertiary_angle_3', 'atom_4', 'tertiary_distance_0', 'd_2_0', 'atom_3', 'cos_c0_c1', 'd_4_2', 'd_5_3', 'd_4_0', 'd_6_3', 'd_3_0', 'dist_H_0_y', 'dist_C_0_x', 'adN1', 'bond_atom', 'd_8_2', 'tertiary_angle_0', 'distC0', 'adC3', 'inv_dist1R', 'd_8_1', 'd_7_1', 'eem_1', 'dist_C_1_y', 'tertiary_distance_3', 'atom_index_1_cycle_size_mean', 'inv_dist0', 'd_2_1', 'dist_H_2_y', 'atom_1_n_bonds', 'tertiary_angle_2', 'd_1_0', 'molecule_atom_index_0_dist_mean_diff', 'dist_C_2_y', 'tertiary_atom_1', 'vander_O.y', 'tertiary_atom_0', 'distC1', 'adC2', 'atom_7', 'max_molecule_atom_0_dist_xyz', 'atom_1_bond_lengths_mean', 'dist_C_4_y', 'd_5_1', 'molecule_type_dist_max', 'd_6_1', 'distance_farthest_0', 'molecule_atom_index_0_dist_min_div', 'tertiary_distance_2', 'atom_5', 'dist_H_0_x', 'cos_c0', 'cos_f0'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_estimators':3000, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['molecule_atom_index_0_dist_min_diff', 'inv_distPR', 'tertiary_angle_1', 'dist_C_0_y', 'tertiary_atom_2', 'd_6_2', 'dist_C_1_x', 'd_4_1', 'dist_N_0_y', 'cos_c1', 'dist_to_type_mean', 'yukawa_H.x', 'cos_f1', 'dist_O_0_x', 'dist_H_3_x', 'dist_C_3_x', 'dist_C_3_y', 'd_7_2', 'dist_O_1_y', 'd_3_2', 'd_5_2', 'dist_C_2_x', 'inv_distPE', 'dist_N_0_x', 'dist_to_type_0_mean', 'molecule_dist_min', 'dist_O_0_y', 'd_4_3', 'dist_to_type_1_mean', 'dist_H_1_x', 'd_3_1', 'dist_H_3_y', 'd_5_0', 'tertiary_distance_1', 'molecule_atom_index_0_dist_max_div', 'inv_distP', 'tertiary_angle_3', 'adC4', 'd_2_0', 'atom_3', 'cos_c0_c1', 'd_4_2', 'd_5_3', 'dist_to_type_std', 'd_4_0', 'd_6_3', 'd_3_0', 'dist_H_0_y', 'dist_C_0_x', 'adN1', 'tertiary_angle_0', 'yukawa_C.x', 'mean_molecule_atom_0_dist_xyz', 'adC3', 'dist_H_1_y', 'dist_H_2_x', 'dist_C_1_y', 'tertiary_distance_3', 'coulomb_H.y', 'd_2_1', 'dist_H_2_y', 'vander_H.y', 'link0', 'tertiary_distance_4', 'vander_C.x', 'tertiary_angle_2', 'd_1_0', 'dist_C_2_y', 'tertiary_atom_1', 'yukawa_H.y', 'dist_H_4_y', 'mean_molecule_atom_1_dist_xyz', 'adC2', 'd_5_1', 'molecule_type_dist_max', 'vander_N.y', 'distance_farthest_0', 'molecule_atom_index_0_dist_min_div', 'vander_H.x', 'tertiary_distance_2', 'atom_5', 'cos_c0', 'vander_O.x', 'cos_f0_f1', 'dist_H_0_x', 'max_distance_y', 'dist_xyz', 'cos_f0'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_estimators':3000, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_C_1_y', 'dist_C_1_x', 'atom_3', 'dist_C_0_y', 'd_4_2', 'dist_H_0_x', 'min_molecule_atom_0_dist_xyz', 'd_4_0', 'd_3_0', 'd_5_3', 'cos_c0_c1', 'adC3', 'tertiary_distance_1', 'cos_c0', 'tertiary_distance_3', 'inv_distPR', 'dist_O_0_y', 'molecule_atom_index_1_dist_min_diff', 'tertiary_atom_1', 'd_5_0', 'd_6_3', 'adC2', 'vander_C.x', 'd_3_2', 'd_6_0', 'dist_N_0_x', 'dist_C_3_y', 'max_molecule_atom_0_dist_xyz', 'molecule_atom_index_0_dist_std_div', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'tertiary_angle_0', 'inv_distP', 'd_4_3', 'dist_O_0_x', 'cos_f0', 'molecule_atom_index_0_dist_std_diff', 'yukawa_H.x', 'd_2_1', 'd_3_1', 'inv_dist0R', 'tertiary_distance_2', 'd_5_1', 'tertiary_atom_0', 'd_4_1', 'atom_1_bond_lengths_mean', 'inv_dist0', 'atom_1_bond_lengths_min'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_estimators':3000, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_C_1_y', 'dist_C_1_x', 'atom_3', 'dist_C_0_y', 'd_4_2', 'dist_H_0_x', 'min_molecule_atom_0_dist_xyz', 'd_4_0', 'd_3_0', 'd_5_3', 'cos_c0_c1', 'adC3', 'tertiary_distance_1', 'cos_c0', 'tertiary_distance_3', 'inv_distPR', 'dist_O_0_y', 'molecule_atom_index_1_dist_min_diff', 'tertiary_atom_1', 'd_5_0', 'd_6_3', 'adC2', 'vander_C.x', 'd_3_2', 'd_6_0', 'dist_N_0_x', 'dist_C_3_y', 'max_molecule_atom_0_dist_xyz', 'molecule_atom_index_0_dist_std_div', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'tertiary_angle_0', 'inv_distP', 'd_4_3', 'dist_O_0_x', 'cos_f0', 'molecule_atom_index_0_dist_std_diff', 'yukawa_H.x', 'd_2_1', 'd_3_1', 'inv_dist0R', 'tertiary_distance_2', 'd_5_1', 'tertiary_atom_0', 'd_4_1', 'atom_1_bond_lengths_mean', 'inv_dist0', 'atom_1_bond_lengths_min'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_estimators':3000, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_C_1_y', 'dist_C_1_x', 'atom_3', 'dist_C_0_y', 'd_4_2', 'dist_H_0_x', 'min_molecule_atom_0_dist_xyz', 'd_4_0', 'd_3_0', 'd_5_3', 'cos_c0_c1', 'adC3', 'tertiary_distance_1', 'cos_c0', 'tertiary_distance_3', 'inv_distPR', 'dist_O_0_y', 'molecule_atom_index_1_dist_min_diff', 'tertiary_atom_1', 'd_5_0', 'd_6_3', 'adC2', 'vander_C.x', 'd_3_2', 'd_6_0', 'dist_N_0_x', 'dist_C_3_y', 'max_molecule_atom_0_dist_xyz', 'molecule_atom_index_0_dist_std_div', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'tertiary_angle_0', 'inv_distP', 'd_4_3', 'dist_O_0_x', 'cos_f0', 'molecule_atom_index_0_dist_std_diff', 'yukawa_H.x', 'd_2_1', 'd_3_1', 'inv_dist0R', 'tertiary_distance_2', 'd_5_1', 'tertiary_atom_0', 'd_4_1', 'atom_1_bond_lengths_mean', 'inv_dist0', 'atom_1_bond_lengths_min'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_estimators':3000, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_to_type_1_mean', 'cos_f0', 'dist_O_0_x', 'atom_1_bond_lengths_min', 'd_5_2', 'cos_c1', 'd_2_1', 'molecule_atom_index_0_dist_min_div', 'atom_index_1_cycle_size_mean', 'd_4_2', 'inv_distPR', 'dist_N_0_x', 'dist_O_0_y', 'adC1', 'adC2', 'tertiary_angle_2', 'dist_C_2_y', 'molecule_atom_index_0_dist_std_diff', 'dist_C_2_x', 'dist_C_3_y', 'd_4_3', 'eem_1', 'molecule_atom_index_1_dist_min_diff', 'atom_1_bond_lengths_mean', 'tertiary_angle_1', 'inv_dist0R', 'd_4_0', 'd_4_1', 'mean_molecule_atom_0_dist_xyz', 'd_2_0', 'd_3_1', 'dist_C_1_x', 'dist_H_0_y', 'dist_C_0_y', 'd_5_3', 'tertiary_distance_3', 'd_3_2', 'dist_H_1_x', 'tertiary_distance_1', 'atom_1_n_bonds', 'd_8_3', 'cos_c0', 'tertiary_distance_2', 'tertiary_angle_0', 'cos_c0_c1', 'molecule_atom_index_0_dist_min_diff', 'yukawa_H.y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_estimators':3000, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
]

In [15]:
def sk_process2(df_train, param, message, df_test=None, trial=None, is_output_feature_importance=False, trial_level=0):

    columns = param['columns']

    assert 'y' in df_train.columns.tolist(), 'y is not in df_train'
    assert 'index' in df_train.columns.tolist(), 'index is not in df_train'
    assert 'index' not in param['columns'], 'index is in features'
    assert 'y' not in param['columns'], 'y is in features'
    assert 'label' not in param['columns'], 'label is in features'
    assert 'group' not in param['columns'], 'group is in features'
    assert (type(trial) == list) | (trial == None), 'trial is neither list nor none'
    assert len(columns) != 0, 'columns size is 0'

    df_test_pred = None
    if type(df_test) == pd.DataFrame:
        assert 'index' in df_test.columns.tolist(), 'index is not in df_test'
        df_test_pred = pd.concat([df_test_pred, df_test[['index']]], axis=1)

    CV = processutil._str2class(param['cv']['cls'])
    MODEL = processutil._str2class(param['model']['cls'])
    if 'scaler' in param:
        SCALER = processutil._str2class(param['scaler']['cls'])
    metric = processutil._str2class(param['metric'])

    history = []
    df_valid_pred = pd.DataFrame()
    df_feature_importances_i_list = []

    # StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
    if 'splits' in param['cv']:
        splits = param['cv']['splits']
    else:
        cv = CV(**param['cv']['init'])
        if param['cv']['cls'] == 'StratifiedKFold':
            assert 'label' in df_train.columns.tolist(), 'label is not in df_train'
            splits = list(cv.split(df_train, df_train['label']))
        elif param['cv']['cls'] == 'GroupKFold':
            assert 'group' in df_train.columns.tolist(), 'group is not in df_train'
            splits = list(cv.split(df_train, groups=df_train['group']))
        else:
            splits = list(cv.split(df_train))

    for fold_n, (train_index, valid_index) in enumerate(splits):

        X_train, X_valid = df_train[columns].values[train_index, :], df_train[columns].values[valid_index, :]
        y_train, y_valid = df_train['y'].values[train_index], df_train['y'].values[valid_index]

        if 'scaler' in param:
            scaler = SCALER(**param['scaler']['init'])
            X_train = scaler.fit_transform(X_train)
            X_valid = scaler.transform(X_valid)

        model = MODEL(**param['model']['init'])
        
        fit_param = param['model']['fit'].copy()
        if 'eval_set' in fit_param:
            fit_param['eval_set'] = [(X_valid, y_valid)]
        
        model.fit(X_train, y_train, **fit_param)

        y_valid_pred = model.predict(X_valid)
        y_train_pred = model.predict(X_train)

        original_index = df_train['index'].values[valid_index]
        df_valid_pred_i = pd.DataFrame \
            ({'index': original_index, 'predict': y_valid_pred, 'fold_n': np.zeros(y_valid_pred.shape[0]) + fold_n})
        df_valid_pred = pd.concat([df_valid_pred, df_valid_pred_i], axis=0)

        if is_output_feature_importance:
            df_feature_importances_i = pd.DataFrame({'feature': columns, 'model_weight': model.feature_importances_})
            df_feature_importances_i = df_feature_importances_i.sort_values(by=['feature'])
            df_feature_importances_i = df_feature_importances_i.reset_index(drop=True)
            perm = PermutationImportance(model, random_state=42).fit(X_valid, y_valid)
            df_feature_importances_i2 = eli5.explain_weights_dfs(perm, feature_names=columns, top=len(columns))['feature_importances']
            df_feature_importances_i2 = df_feature_importances_i2.sort_values(by=['feature'])
            df_feature_importances_i2 = df_feature_importances_i2.reset_index(drop=True)
            df_feature_importances_i = pd.merge(df_feature_importances_i, df_feature_importances_i2, on='feature')
            df_feature_importances_i_list.append(df_feature_importances_i)

        if type(df_test) == pd.DataFrame:
            X_test = df_test[columns].values
            if 'scaler' in param:
                X_test = scaler.transform(X_test)
            y_test_pred = model.predict(X_test)
            df_test_pred_i = pd.DataFrame({fold_n: y_test_pred})
            df_test_pred = pd.concat([df_test_pred, df_test_pred_i], axis=1)

        history.append \
            ({'fold_n': fold_n, 'train': metric(y_train, y_train_pred, group=df_train['group']), 'valid': metric(y_valid, y_valid_pred, group=df_train['group'])})

    df_his = pd.DataFrame(history)

    df_feature_importances = None
    if is_output_feature_importance:
        df_feature_importances = df_feature_importances_i_list[0]
        for idx, df_feature_importances_i in enumerate(df_feature_importances_i_list[1:]):
            df_feature_importances = pd.merge(df_feature_importances, df_feature_importances_i, on='feature', suffixes=('', idx + 1))

    df_valid_pred = df_valid_pred.sort_values(by=['index'])
    df_valid_pred = df_valid_pred.reset_index(drop=True)

    if type(df_test) == pd.DataFrame:
        df_test_pred = df_test_pred.sort_values(by=['index'])
        df_test_pred = df_test_pred.reset_index(drop=True)

    if type(trial) == list:
        datetime_ = datetime.datetime.now()
        val_metric_mean = np.mean(df_his.valid)
        val_metric_std = np.std(df_his.valid)
        train_metric_mean = np.mean(df_his.train)
        train_metric_std = np.std(df_his.train)

        trial_i_d_ = {'datetime': datetime_, 'message': message, 'val_metric_mean': val_metric_mean,
                      'train_metric_mean': train_metric_mean, 'val_metric_std': val_metric_std, 'train_metric_std': train_metric_std,
                      'trn_val_metric_diff': val_metric_mean - train_metric_mean,
                      'df_feature_importances': df_feature_importances ,'param': param.copy(),
                      'nfeatures': len(columns)}
        if trial_level > 0:
            trial_i_d_ = {'df_his': df_his, 'df_valid_pred': df_valid_pred, 'df_test_pred': df_test_pred, **trial_i_d_}
        trial.append(trial_i_d_)

    return df_his, df_feature_importances, df_valid_pred, df_test_pred

In [ ]:
mytrial=[]
df_his, df_feature_importances, df_valid_pred, df_test_pred = pd.DataFrame(), pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
for t in  df_train.type.unique().tolist():
    
    
    columns = columns_list[t]['columns']
    param['model']['init']['input_dim'] = len(columns)
    param['columns'] = columns
    
    df_his_i, df_feature_importances_i, df_valid_pred_i, df_test_pred_i =  sk_process2(df_train[df_train['type']==t].reset_index(drop=True), param, f'modeling for {t}', df_test=None, trial=mytrial, is_output_feature_importance=False, trial_level=0)
    df_his = pd.concat([df_his, df_his_i], axis=0)
    df_feature_importances = pd.concat([df_feature_importances, df_feature_importances_i], axis=0)
    df_valid_pred = pd.concat([df_valid_pred, df_valid_pred_i], axis=0)
    df_test_pred = pd.concat([df_test_pred, df_test_pred_i], axis=0)
    break

df_valid_pred = df_valid_pred.sort_values(by=['index']).reset_index(drop=True)
df_test_pred = df_test_pred.sort_values(by=['index']).reset_index(drop=True)

In [ ]:
df_trial = pd.DataFrame(mytrial)
df_trial['trn_val_metric_diff_rate'] = df_trial['trn_val_metric_diff'] / df_trial['train_metric_mean']
df_trial['log_val_mae'] = df_trial['val_metric_mean'].apply(lambda x : np.log(x))
print(mean_absolute_error(df_valid_pred.sort_values(by=['index']).reset_index(drop=True).predict.values, df_train.reset_index(drop=True).y.values))
df_trial[['datetime', 'message', 'nfeatures', 'train_metric_mean', 'val_metric_mean', 'trn_val_metric_diff', 'trn_val_metric_diff_rate', 'message', 'log_val_mae']]